In [59]:
import uuid, sys, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
import math
import random

from sklearn import metrics
from scipy import stats

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# import torch
# torch.cuda.set_device(0)  # 0 == "first visible" -> actually GPU 2 on the node
# print(torch.cuda.get_device_name(0))

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
import pytorch_lightning as pl
from torch.optim import AdamW

torch.manual_seed(0)

from accelerate import Accelerator

import matplotlib.pyplot as plt
import seaborn as sns

from Levenshtein import distance as Ldistance

import training_utils.dataset_utils as data_utils
import training_utils.partitioning_utils as pat_utils

import importlib
import training_utils.train_utils as train_utils
importlib.reload(train_utils)

<module 'training_utils.train_utils' from '/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts/training_utils/train_utils.py'>

In [60]:
import wandb
wandb.login()

True

In [61]:
os.chdir("/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts")
# print(os.getcwd())

print("PyTorch:", torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print("Current location:", os.getcwd())

PyTorch: 2.5.1
Using device: cuda
Current location: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts


In [62]:
# Model parameters
memory_verbose = False
use_wandb = True # Used to track loss in real-time without printing
model_save_steps = 1
train_frac = 1.0
test_frac = 1.0

embedding_dimension = 1152 # 1280 | 960 | 1152
number_of_recycles = 2
padding_value = -5000

# batch_size = 20
learning_rate = 2e-5
EPOCHS = 15

In [63]:
## Model Class
### MiniClip 
def gaussian_kernel(x, sigma):
    return np.exp(-x**2 / (2 * sigma**2))

def transform_vector(vector, sigma):

    interacting_indices = np.where(vector == 1)[0]   # positions where vector == 1
    transformed_vector = np.zeros_like(vector, dtype=float)
    
    for i in range(len(vector)):
        if vector[i] == 0:
            distances = np.abs(interacting_indices - i)   # distance to all "1"s
            min_distance = np.min(distances)              # closest "1"
            transformed_vector[i] = gaussian_kernel(min_distance, sigma)
        else:
            transformed_vector[i] = 1.0
    return transformed_vector

def safe_shuffle(n, device):
    shuffled = torch.randperm(n, device=device)
    while torch.any(shuffled == torch.arange(n, device=device)):
        shuffled = torch.randperm(n, device=device)
    return shuffled

def create_key_padding_mask(embeddings, padding_value=-5000, offset=10):
    return (embeddings < (padding_value + offset)).all(dim=-1)

def create_mean_of_non_masked(embeddings, padding_mask):
    # Use masked select and mean to compute the mean of non-masked elements
    # embeddings should be of shape (batch_size, seq_len, features)
    seq_embeddings = []
    for i in range(embeddings.shape[0]): # looping over all batch elements
        non_masked_embeddings = embeddings[i][~padding_mask[i]] # shape [num_real_tokens, features]
        if len(non_masked_embeddings) == 0:
            print("You are masking all positions when creating sequence representation")
            sys.exit(1)
        mean_embedding = non_masked_embeddings.mean(dim=0) # sequence is represented by the single vecotr [1152] [features]
        seq_embeddings.append(mean_embedding)
    return torch.stack(seq_embeddings)

class MiniCLIP_w_transformer_crossattn(pl.LightningModule):

    def __init__(self, padding_value = -5000, embed_dimension=1152, num_recycles=1):

        super().__init__()
        self.num_recycles = num_recycles # how many times you iteratively refine embeddings with self- and cross-attention (ALPHA-Fold-style recycling).
        self.padding_value = padding_value
        self.embed_dimension = embed_dimension

        self.logit_scale = nn.Parameter(torch.tensor(math.log(1/0.07)))  # ~CLIP init

        self.transformerencoder =  nn.TransformerEncoderLayer(
            d_model=self.embed_dimension,
            nhead=8,
            dropout=0.1,
            batch_first=True,
            dim_feedforward=self.embed_dimension
            )
 
        self.norm = nn.LayerNorm(self.embed_dimension)  # For residual additions

        self.cross_attn = nn.MultiheadAttention(
            embed_dim=self.embed_dimension,
            num_heads=8,
            dropout=0.1,
            batch_first=True
        )

        self.prot_embedder = nn.Sequential(
            nn.Linear(self.embed_dimension, 640),
            nn.ReLU(),
            nn.Linear(640, 320),
        )
        
    def forward(self, pep_input, prot_input, label=None, pep_int_mask=None, prot_int_mask=None, int_prob=None, mem_save=True): # , pep_tokens, prot_tokens

        pep_mask = create_key_padding_mask(embeddings=pep_input, padding_value=self.padding_value)
        prot_mask = create_key_padding_mask(embeddings=prot_input, padding_value=self.padding_value)
 
        # Initialize residual states
        pep_emb = pep_input.clone()
        prot_emb = prot_input.clone()
 
        for _ in range(self.num_recycles):

            # Transformer encoding with residual
            pep_trans = self.transformerencoder(self.norm(pep_emb), src_key_padding_mask=pep_mask)
            prot_trans = self.transformerencoder(self.norm(prot_emb), src_key_padding_mask=prot_mask)

            # Cross-attention with residual
            pep_cross, _ = self.cross_attn(query=self.norm(pep_trans), key=self.norm(prot_trans), value=self.norm(prot_trans), key_padding_mask=prot_mask)
            prot_cross, _ = self.cross_attn(query=self.norm(prot_trans), key=self.norm(pep_trans), value=self.norm(pep_trans), key_padding_mask=pep_mask)
            
            # Additive update with residual connection
            pep_emb = pep_emb + pep_cross  
            prot_emb = prot_emb + prot_cross

        pep_seq_coding = create_mean_of_non_masked(pep_emb, pep_mask)
        prot_seq_coding = create_mean_of_non_masked(prot_emb, prot_mask)
        
        # Use self-attention outputs for embeddings
        pep_seq_coding = F.normalize(self.prot_embedder(pep_seq_coding))
        prot_seq_coding = F.normalize(self.prot_embedder(prot_seq_coding))
 
        if mem_save:
            torch.cuda.empty_cache()
        
        scale = torch.exp(self.logit_scale).clamp(max=100.0)
        logits = scale * (pep_seq_coding * prot_seq_coding).sum(dim=-1) # Dot-Product for comparison
        
        return logits
    

    def training_step(self, batch, device):
        # Passing the sequences to the models
        embedding_pep = batch[0]
        embedding_prot = batch[1]
        binder_label = batch[2]
 
        embedding_pep = embedding_pep.to(device)
        embedding_prot = embedding_prot.to(device)
        binder_label = binder_label.to(device)

        logits = self.forward(embedding_pep, embedding_prot)
        binder_labels = binder_label.view_as(logits)  
        loss = F.binary_cross_entropy_with_logits(logits, binder_labels)
 
        torch.cuda.empty_cache()
        
        return loss
    
    def validation_step(self, batch, device):
        # Predict on random batches of training batch size
        embedding_pep, embedding_prot, binder_label = batch
 
        # Move to device
        embedding_pep  = embedding_pep.to(device)
        embedding_prot = embedding_prot.to(device)
        binder_label = binder_label.to(device).float() 

        with torch.no_grad():
            logits = self.forward(embedding_pep, embedding_prot)   # shape [B]
            binder_labels = binder_label.view_as(logits)  
            loss = F.binary_cross_entropy_with_logits(logits, binder_labels)
            
        return float(loss.item()), logits, binder_labels

In [64]:
## Output path
trained_model_dir = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts"

## Embeddings paths
binders_embeddings = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/meta_analysis/binders_embeddings"
targets_embeddings = "/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/meta_analysis/targets_embeddings"

# ## Training variables
runID = uuid.uuid4()

def print_mem_consumption():
    # 1. Total memory available on the GPU (device 0)
    t = torch.cuda.get_device_properties(0).total_memory
    # 2. How much memory PyTorch has *reserved* from CUDA
    r = torch.cuda.memory_reserved(0)
    # 3. How much of that reserved memory is actually *used* by tensors
    a = torch.cuda.memory_allocated(0)
    # 4. Reserved but not currently allocated (so “free inside PyTorch’s pool”)
    f = r - a

    print("Total memory: ", t/1e9)      # total VRAM in GB
    print("Reserved memory: ", r/1e9)   # PyTorch’s reserved pool in GB
    print("Allocated memory: ", a//1e9) # actually in use (integer division)
    print("Free memory: ", f/1e9)       # slack in the reserved pool in GB

#### Loading data frame

In [65]:
### Loading the dataset
interaction_df = pd.read_csv("../data/meta_analysis/interaction_df_metaanal.csv", index_col = 0).drop(columns = ["binder_id", "target_id"]).rename(columns={
    "A_seq" : "binder_seq",
    "B_seq" : "target_seq"
})

all_targets = interaction_df.target_id_mod.unique()
binder_nonbinder = interaction_df.binder.value_counts()
target_binder_nonbinder_Dict = dict(interaction_df.groupby("target_id_mod")["binder"].value_counts())
sorted_items = sorted(target_binder_nonbinder_Dict.items(), key=lambda kv: kv[1], reverse=True)

# %%
# Annotating each observation with a weight corresponding to whether it is considered a binder or not

### Weights for binder/non-binders
N_bins = len(interaction_df["binder"].value_counts())
pr_class_uniform_weight = 1 / N_bins
pr_class_weight_informed_with_size_of_bins = pr_class_uniform_weight  / interaction_df["binder"].value_counts()
pr_class_weight_informed_with_size_of_bins = pr_class_weight_informed_with_size_of_bins.to_dict()
interaction_df["class_weight"] = interaction_df.binder.apply(lambda x: pr_class_weight_informed_with_size_of_bins[x])
# binder_nonbinder_weights_Dict = dict(zip(interaction_df["target_binder_ID"], interaction_df["class_weight"]))

### Weights for target
N_bins = len(interaction_df["target_id_mod"].value_counts())
pr_class_uniform_weight = 1 / N_bins
pr_class_weight_informed_with_size_of_bins = pr_class_uniform_weight  / interaction_df["target_id_mod"].value_counts()
pr_class_weight_informed_with_size_of_bins = pr_class_weight_informed_with_size_of_bins.to_dict()
interaction_df["target_weight"] = interaction_df.target_id_mod.apply(lambda x: pr_class_weight_informed_with_size_of_bins[x])

### Combined weights
interaction_df["combined_weight"] = interaction_df["class_weight"]*interaction_df["target_weight"] 

In [66]:
interaction_df

,binder_chain,target_chains,binder,binder_seq,target_seq,target_id_mod,target_binder_ID,class_weight,target_weight,combined_weight
0,A,"[""B""]",False,LDFIVFAGPEKAIKFYKEMAKRNLEVKIWIDGDWAVVQVK,ANPYISVANIMLQNYVKQREKYNYDTLKEQFTFIKNASTSIVYMQF...,VirB8,VirB8_1,0.000159,0.000631,1.004956e-07
1,A,"[""B""]",False,SEQDETMHRIVRSVIQHAYKHNDEMAEYFAQNAAEIYKEQNKSEEA...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2,FGFR2_1,0.000159,0.000029,4.686322e-09
2,A,"[""B""]",False,DYKQLKKHATKLLELAKKDPSSKRDLLRTAASYANKVLFEDSDPRA...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2,FGFR2_2,0.000159,0.000029,4.686322e-09
3,A,"[""B""]",False,DEKEELERRANRVAFLAIQIQNEEYHRILAELYVQFMKAAENNDTE...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2,FGFR2_3,0.000159,0.000029,4.686322e-09
4,A,"[""B""]",False,PDNKEKLMSIAVQLILRINEAARSEEQWRYANRAAFAAVEASSGSD...,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,FGFR2,FGFR2_4,0.000159,0.000029,4.686322e-09
...,...,...,...,...,...,...,...,...,...,...
3527,A,"[""B""]",False,DLRKYAAELVDRLAEKYNLDSDQYNALVRLASELVWQGKSKEEIEK...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra,IL2Ra_62,0.000159,0.000947,1.507433e-07
3528,A,"[""B""]",False,SKEEIKKEAEELIEELKKKGYNLPLRILEFALKEIEETNSEKYYEQ...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra,IL2Ra_63,0.000159,0.000947,1.507433e-07
3529,A,"[""B""]",False,SPEYKKFLELIKEAEAARKAGDLDKAKELLEKALELAKKMKAKSLI...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra,IL2Ra_64,0.000159,0.000947,1.507433e-07
3530,A,"[""B""]",False,DPLLAYKLLKLSQKALEKAYAEDRERAEELLEEAEAALRSLGDEAG...,ELCDDDPPEIPHATFKAMAYKEGTMLNCECKRGFRRIKSGSLYMLC...,IL2Ra,IL2Ra_65,0.000159,0.000947,1.507433e-07


# 5-fold CV training + weighting of samples

In [67]:
clusters = [
    ["VirB8"], ["FGFR2"], ["IL7Ra"], ["InsulinR"],
    ["EGFR", "EGFR_2", "EGFR_3"],     # keep together
    ["SARS_CoV2_RBD"], ["Pdl1"], ["TrkA"], ["IL10Ra"],
    ["LTK"], ["Mdm2"],
    ["sntx", "sntx_2"],               # keep together
    ["IL2Ra"],
]

random.Random(0).shuffle(clusters)
folds = np.array_split(np.array(clusters, dtype=object), 5)   # list of np arrays
targets_folds = []
for f in folds:
    flat = []
    for group in f:
        flat.extend(group)
    targets_folds.append(flat)

def build_cv_splits(targets_folds):
    val_folds, train_folds = [], []
    K = len(targets_folds)
    for i in range(K):
        val_targets = list(targets_folds[i])  # copy
        train_targets = [t for j, fold in enumerate(targets_folds) if j != i for t in fold]
        val_folds.append(val_targets)
        train_folds.append(train_targets)
    return val_folds, train_folds

val_folds, train_folds = build_cv_splits(targets_folds)
cv_splits = list(zip(val_folds, train_folds))

In [68]:
for idx in range(len(cv_splits)):
    val_targets = cv_splits[idx][0]
    vals = len(interaction_df[interaction_df.target_id_mod.isin(val_targets)])
    trains = len(interaction_df) - vals
    print(f"Fold {idx+1} : training instances : {trains}, validation instances : {vals}")

Fold 1 : training instances : 1280, validation instances : 2252
Fold 2 : training instances : 3213, validation instances : 319
Fold 3 : training instances : 3265, validation instances : 267
Fold 4 : training instances : 2999, validation instances : 533
Fold 5 : training instances : 3371, validation instances : 161


In [89]:
for idx in range(len(cv_splits)):
    val_targets = cv_splits[idx][0]
    train_targets = cv_splits[idx][1]
    print(f"Fold {idx+1} : validation targets : {val_targets}")

Fold 1 : validation targets : ['FGFR2', 'Mdm2', 'LTK']
Fold 2 : validation targets : ['SARS_CoV2_RBD', 'sntx', 'sntx_2', 'IL7Ra']
Fold 3 : validation targets : ['InsulinR', 'TrkA', 'IL10Ra']
Fold 4 : validation targets : ['EGFR', 'EGFR_2', 'EGFR_3', 'VirB8']
Fold 5 : validation targets : ['IL2Ra', 'Pdl1']


In [104]:
interaction_df[interaction_df.target_id_mod == "FGFR2"].target_seq.count()

2123

In [99]:
interaction_df[interaction_df.target_id_mod == "EGFR_2"].target_seq.unique()

array(['LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGSCQKCDPSCPNGSCWGAGEENCQKLTKIICAQQCSGRCRGKSPSDCCHNQCAAGCTGPRESDCLVCRKFRDEATCKDTCPPLMLYNPTTYQMDVNPEGKYSFGATCVKKCPRNYVVTDHGSCVRACGADSYEMEEDGVRKCKKCEGPCRKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCVSCRNVSRGRECVDKCNLLEGEPREFVENSECIQCHPECLPQAMNITCTGRGPDNCIQCAHYIDGPHCVKTCPAGVMGENNTLVWKYADAGHVCHLCHPNCTYGCTGPGLEGCPTNGPKIPS'],
      dtype=object)

In [100]:
interaction_df[interaction_df.target_id_mod == "EGFR"].target_seq.unique()

array(['RKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGDSFTHTPPLDPQELDILKTVKEITGFLLIQAWPENRTDLHAFENLEIIRGRTKQHGQFSLAVVSLNITSLGLRSLKEISDGDVIISGNKNLCYANTINWKKLFGTSGQKTKIISNRGENSCKATGQVCHALCSPEGCWGPEPRDCV'],
      dtype=object)

In [102]:
interaction_df[interaction_df.target_id_mod == "EGFR_3"].target_seq.unique()

array(['VCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYVQRNYDLSFLKTIQEVAGYVLIALNTVERIPLENLQIIRGNMYYENSYALAVLSNYDANKTGLKELPMRNLQEILHGAVRFSNNPALCNVESIQWRDIVSSDFLSNMSMDFQNHLGS'],
      dtype=object)

#### Creating separate targets/ binder dataframes

In [69]:
# Targets df
target_df = interaction_df[["target_id_mod","target_seq"]].rename(columns={"target_seq":"sequence", "target_id_mod" : "ID"})
target_df["seq_len"] = target_df["sequence"].apply(len)
target_df = target_df.drop_duplicates(subset=["ID","sequence"])
target_df = target_df.set_index("ID")

# Binders df
binder_df = interaction_df[["target_binder_ID","binder_seq", "binder", "class_weight", "target_weight", "combined_weight"]].rename(columns={"binder_seq":"sequence", "target_binder_ID" : "ID", "binder" : "label"})
binder_df["seq_len"] = binder_df["sequence"].apply(len)
binder_df = binder_df.set_index("ID")

# Interaction Dict
interaction_Dict = dict(enumerate(zip(interaction_df["target_id_mod"], interaction_df["target_binder_ID"]), start=1))

# Printing dataframes for review
binder_df

,sequence,label,class_weight,target_weight,combined_weight,seq_len
ID,,,,,,
VirB8_1,LDFIVFAGPEKAIKFYKEMAKRNLEVKIWIDGDWAVVQVK,False,0.000159,0.000631,1.004956e-07,40
FGFR2_1,SEQDETMHRIVRSVIQHAYKHNDEMAEYFAQNAAEIYKEQNKSEEA...,False,0.000159,0.000029,4.686322e-09,62
FGFR2_2,DYKQLKKHATKLLELAKKDPSSKRDLLRTAASYANKVLFEDSDPRA...,False,0.000159,0.000029,4.686322e-09,61
FGFR2_3,DEKEELERRANRVAFLAIQIQNEEYHRILAELYVQFMKAAENNDTE...,False,0.000159,0.000029,4.686322e-09,64
FGFR2_4,PDNKEKLMSIAVQLILRINEAARSEEQWRYANRAAFAAVEASSGSD...,False,0.000159,0.000029,4.686322e-09,64
...,...,...,...,...,...,...
IL2Ra_62,DLRKYAAELVDRLAEKYNLDSDQYNALVRLASELVWQGKSKEEIEK...,False,0.000159,0.000947,1.507433e-07,55
IL2Ra_63,SKEEIKKEAEELIEELKKKGYNLPLRILEFALKEIEETNSEKYYEQ...,False,0.000159,0.000947,1.507433e-07,56
IL2Ra_64,SPEYKKFLELIKEAEAARKAGDLDKAKELLEKALELAKKMKAKSLI...,False,0.000159,0.000947,1.507433e-07,56


In [70]:
target_df

,sequence,seq_len
ID,,
VirB8,ANPYISVANIMLQNYVKQREKYNYDTLKEQFTFIKNASTSIVYMQF...,138
FGFR2,RSPHRPILQAGLPANASTVVGGDVEFVCKVYSDAQPHIQWIKHVPY...,101
IL7Ra,DYSFSCYSQLEVNGSQHSLTCAFEDPDVNTTNLEFEICGALVEVKC...,193
InsulinR,EVCPGMDIRNNLTRLHELENCSVIEGHLQILLMFKTRPEDFRDLSF...,150
EGFR,RKVCNGIGIGEFKDSLSINATNIKHFKNCTSISGDLHILPVAFRGD...,191
SARS_CoV2_RBD,TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFK...,195
Pdl1,NAFTVTVPKDLYVVEYGSNMTIECKFPVEKQLDLAALIVYWEMEDK...,115
EGFR_2,LEEKKVCQGTSNKLTQLGTFEDHFLSLQRMFNNCEVVLGNLEITYV...,621
TrkA,VSFPASVQLHTAVEMHHWCIPFSVDGQPAPSLRWLFNGSVLNETSF...,101


#### Creating separate targets/ binder dataframes (for validation/ training)

In [71]:
class CLIP_meta_analysis_dataset(Dataset):

    def __init__(self, sequence_df, esm_encoding_paths, embedding_dim=1152, padding_value=-5000):

        super(CLIP_meta_analysis_dataset, self).__init__()

        self.sequence_df = sequence_df # target/binder_df
        self.max_length = sequence_df["seq_len"].max()
        self.sequence_df["index_num"] = np.arange(len(self.sequence_df))
        # print(self.sequence_df)
        self.esm_encoding_paths = esm_encoding_paths
        num_samples = len(self.sequence_df)
        
        self.x = torch.full((num_samples, self.max_length, embedding_dim), padding_value, dtype=torch.float32)

        self.accessions = self.sequence_df.index.astype(str).tolist()
        self.name_to_row = {name: i for i, name in enumerate(self.accessions)}
        
        # Load embeddings into the pre-allocated tensor
        all_acc_list = self.accessions
        all_acc_loaded_list = []

        iterator = tqdm(all_acc_list, position=0, leave=True, total=num_samples, desc="# Reading in ESM-embeddings from folder")
        for i, accession in enumerate(iterator):
            npy_path = os.path.join(esm_encoding_paths, f"{accession}.npy")
            try:
                embd = np.load(npy_path)[0]
                length_to_pad = self.max_length - len(embd)
                if length_to_pad > 0:
                    zero_padding = np.ones((length_to_pad, embd.shape[1])) * padding_value
                    padded_array = np.concatenate((embd, zero_padding), axis=0)
                else:
                    padded_array = embd[:self.max_length] 
                self.x[i] = torch.tensor(padded_array, dtype=torch.float32)
                all_acc_loaded_list.append(accession)
            except FileNotFoundError as e:
                raise FileNotFoundError(f"Embedding file {accession}.npy not found.")
            
        missing = sorted(set(all_acc_list) - set(all_acc_loaded_list))
        if missing:
            raise FileNotFoundError(
                f"Missing {len(missing)} embedding files in '{esm_encoding_paths}'. "
                f"Examples: {missing}")
          
    def __len__(self):
        return int(self.x.shape[0])

    def __getitem__(self, idx):
        return self.x[idx]
    
    # add a helper:
    def get_by_name(self, name: str):
        return self.x[self.name_to_row[name]]

targets_dataset = CLIP_meta_analysis_dataset(target_df, esm_encoding_paths="/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/meta_analysis/targets_embeddings", embedding_dim=1152)
binders_dataset = CLIP_meta_analysis_dataset(binder_df, esm_encoding_paths="/zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/data/meta_analysis/binders_embeddings", embedding_dim=1152)

# Reading in ESM-embeddings from folder: 100%|█████████████████████████████████████████████████████████| 3532/3532 [00:06<00:00, 584.20it/s]


In [72]:
binders_dataset[12]

tensor([[-1.3036e-02, -2.4448e-03, -6.8176e-03,  ...,  5.2219e-03,
          1.3421e-02,  1.3365e-02],
        [ 3.5381e-03, -1.6180e-04, -2.7373e-02,  ...,  3.8627e-02,
          1.8391e-02,  2.9930e-02],
        [-5.4220e-03, -4.6685e-03, -5.6514e-02,  ...,  2.7079e-02,
          3.4216e-02,  9.9046e-03],
        ...,
        [-5.0000e+03, -5.0000e+03, -5.0000e+03,  ..., -5.0000e+03,
         -5.0000e+03, -5.0000e+03],
        [-5.0000e+03, -5.0000e+03, -5.0000e+03,  ..., -5.0000e+03,
         -5.0000e+03, -5.0000e+03],
        [-5.0000e+03, -5.0000e+03, -5.0000e+03,  ..., -5.0000e+03,
         -5.0000e+03, -5.0000e+03]])

In [73]:
torch.equal(targets_dataset[0], targets_dataset.get_by_name("VirB8"))

True

In [74]:
torch.equal(binders_dataset[0], binders_dataset.get_by_name("VirB8_1"))

True

In [75]:
def binder_to_target_name(bname: str) -> str:
    parts = bname.split("_")
    if bname.startswith("SARS"):
        return "SARS_CoV2_RBD"
    if len(parts) == 3:
        return f"{parts[0]}_{parts[1]}"
    return parts[0]

def binder_target_label(targets_dataset, binders_dataset, binder_ids, interaction_df, stack=True):
    
    listof_bindertargetlabel = []
    
    for bname in binder_ids:
        tname = binder_to_target_name(bname)

        # get embeddings by name
        b_emb = binders_dataset.get_by_name(bname)
        t_emb = targets_dataset.get_by_name(tname)

        # get label from df
        s = interaction_df.loc[interaction_df['target_binder_ID'] == bname, 'binder']
        # if s.empty:
        #     raise ValueError(f"No label found in interaction_df for binder id '{bname}'")
        lbl = torch.tensor(float(s.iat[0]), dtype=torch.float32)

        listof_bindertargetlabel.append((b_emb, t_emb, lbl))

    return listof_bindertargetlabel
    
validation_data_5clusters = []
training_data_5clusters = []

for i, split in enumerate(cv_splits):
    validation, training = split[0], split[1]
    
    validation_binders = interaction_df.loc[interaction_df["target_id_mod"].isin(validation), "target_binder_ID"].tolist()
    training_binders = interaction_df.loc[interaction_df["target_id_mod"].isin(training), "target_binder_ID"].tolist()

    listof_bindertargetlabel = binder_target_label(targets_dataset, binders_dataset, validation_binders, interaction_df)
    validation_data_5clusters.append(listof_bindertargetlabel)
    
    listof_bindertargetlabel = binder_target_label(targets_dataset, binders_dataset, training_binders, interaction_df)
    training_data_5clusters.append(listof_bindertargetlabel)

In [76]:
# len(validation_data_5clusters)
# >> 5 # 5 folds
# len(validation_data_5clusters[0])
# >> 249 # number of  instances per fold "1" used ofr validation
# len(validation_data_5clusters[0][0])
# >> 3 # binder_emb, target_emb, label

In [77]:
for i in range(len(validation_data_5clusters)):
    print(f"Run {i+1} : len(val_dataset) : {len(validation_data_5clusters[i])}, len(train_dataset) : {len(training_data_5clusters[i])}")

Run 1 : len(val_dataset) : 2252, len(train_dataset) : 1280
Run 2 : len(val_dataset) : 319, len(train_dataset) : 3213
Run 3 : len(val_dataset) : 267, len(train_dataset) : 3265
Run 4 : len(val_dataset) : 533, len(train_dataset) : 2999
Run 5 : len(val_dataset) : 161, len(train_dataset) : 3371


### Loading pretrained model for finetuning

In [78]:
ckpt_path = '../PPI_PLM/models/CLIP_no_structural_information/a1d0549b-3f90-4ce2-b795-7bca2276cb07_checkpoint_4/a1d0549b-3f90-4ce2-b795-7bca2276cb07_checkpoint_epoch_4.pth'
checkpoint = torch.load(ckpt_path, weights_only=False, map_location="cpu")
# print(list(checkpoint["model_state_dict"]))
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = MiniCLIP_w_transformer_crossattn()
model.load_state_dict(checkpoint['model_state_dict'])
torch.cuda.empty_cache()  # frees cached blocks (not live tensors)
device = torch.device("cuda:0")
model.to(device)
# model.train()

MiniCLIP_w_transformer_crossattn(
  (transformerencoder): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=1152, out_features=1152, bias=True)
    )
    (linear1): Linear(in_features=1152, out_features=1152, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=1152, out_features=1152, bias=True)
    (norm1): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (norm): LayerNorm((1152,), eps=1e-05, elementwise_affine=True)
  (cross_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=1152, out_features=1152, bias=True)
  )
  (prot_embedder): Sequential(
    (0): Linear(in_features=1152, out_features=640, bias=True)
    (1): ReLU()
    (2): Linear(in_features=640, out_features=32

### Loading training and validation datasets (DataLoaders)

In [79]:
all_binders = interaction_df["target_binder_ID"].tolist()

# all dataset: binder_enb, target_emb, label
ALL_btl = binder_target_label(targets_dataset, binders_dataset, all_binders, interaction_df)

val_targets = cv_splits[0][0]
train_targets = cv_splits[0][1]

# indexes of validation binders
val_idx = interaction_df.loc[interaction_df.target_id_mod.isin(val_targets)].index.tolist()

# weights of validation binders
# val_weights = interaction_df.loc[interaction_df.target_id_mod.isin(val_targets), "class_weight"].tolist()

# validation dataset : binder_emb, target_emb, label
val_binders = [ALL_btl[idx] for idx in val_idx]

# indexes of training binders
train_idx = interaction_df.loc[interaction_df.target_id_mod.isin(train_targets)].index.tolist()

# weights of training binders
train_weights_class = interaction_df.loc[interaction_df.target_id_mod.isin(train_targets), "class_weight"].tolist()
train_weights_target = interaction_df.loc[interaction_df.target_id_mod.isin(train_targets), "target_weight"].tolist()
train_weights_combined = interaction_df.loc[interaction_df.target_id_mod.isin(train_targets), "combined_weight"].tolist()

# training dataset : binder_emb, target_emb, label
train_binders = [ALL_btl[idx] for idx in train_idx]

#### no_weight (comment out if other weighting is used) CVsplit1

In [80]:
# train_loader   = DataLoader(train_binders, batch_size=32, shuffle=True)

#### class_weight (comment out if other weighting is used) CVsplit1

In [81]:
train_sampler = WeightedRandomSampler(weights=train_weights_class, num_samples=len(train_binders), replacement=True)
train_loader   = DataLoader(train_binders,   batch_size=32, sampler=train_sampler)

#### target_weight (comment out if other weighting is used) CVsplit1

In [82]:
# train_sampler = WeightedRandomSampler(weights=train_weights_target, num_samples=len(train_binders), replacement=True)
# train_loader   = DataLoader(train_binders,   batch_size=32, sampler=train_sampler)

#### combined_weight (comment out if other weighting is used) CVsplit1

In [83]:
# train_sampler = WeightedRandomSampler(weights=train_weights_combined, num_samples=len(train_binders), replacement=True)
# train_loader   = DataLoader(train_binders,   batch_size=32, sampler=train_sampler)

In [84]:
val_loader   = DataLoader(val_binders,   batch_size=32, shuffle=True)

# interaction_Dict = dict(enumerate(zip(interaction_df["target_id_mod"], interaction_df["target_binder_ID"]), start=1))
# ### Creating the DataLoaders
# train_weights = training_dataset._get_observation_weights()
# val_weights = validation_dataset._get_observation_weights()

batch = next(iter(val_loader))
print(f"Shape of the binders embeddings : {batch[0].shape}\n")
print(f"Shape of the targets embeddings : {batch[1].shape}\n")
print(f"Labels (0 - non-binder, 1 - binder) :{(batch[2].numpy())}\n")

with torch.no_grad():
    # batch = next(iter(train_loader))
    # loss = model.training_step(batch,device)
    # print(loss.cpu().numpy())
    batch = next(iter(val_loader))
    loss, auroc, aupr = model.validation_step(batch,device)
    print(loss)

Shape of the binders embeddings : torch.Size([32, 250, 1152])

Shape of the targets embeddings : torch.Size([32, 621, 1152])

Labels (0 - non-binder, 1 - binder) :[0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 1. 0. 0.]

1.1647932529449463


In [85]:
import gc, torch
# del obj  # any large temps you created in the cell
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()

# print_mem_consumption()
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum(p.numel() for p in model.parameters() if p.requires_grad)

torch.cuda.empty_cache()
print_mem_consumption()

Total memory:  42.405855232
Reserved memory:  0.211812352
Allocated memory:  0.0
Free memory:  0.017884672


### Training loop

In [86]:
def batch(iterable, n=20):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

class TrainWrapper_MetaAnal():

    def __init__(self, model, training_loader, validation_loader, test_dataset, 
                 optimizer, EPOCHS, runID, device, test_indexes_for_auROC=None,
                 auROC_batch_size=18, model_save_steps=False, model_save_path=False, 
                 v=False, wandb_tracker=False, split_id=None):
        
        self.model = model 
        self.training_loader = training_loader
        self.validation_loader = validation_loader
        self.EPOCHS = EPOCHS
        self.wandb_tracker = wandb_tracker
        self.model_save_steps = model_save_steps
        self.verbose = v
        self.split_id = split_id
        self.best_vloss = 1e09
        self.optimizer = optimizer
        self.runID = runID
        self.trained_model_dir = model_save_path
        self.print_frequency_loss = 1
        self.device = device
        self.test_indexes_for_auROC = test_indexes_for_auROC
        self.auROC_batch_size = auROC_batch_size
        self.test_dataset = test_dataset

    def train_one_epoch(self):

        self.model.train()
        running_loss = 0.0

        for batch_data in tqdm(self.training_loader, total=len(self.training_loader), desc="Running through epoch"):

            if batch_data[0].size(0) == 1:
                continue
            
            self.optimizer.zero_grad()
            loss = self.model.training_step(batch_data, self.device)
            loss.backward()
            self.optimizer.step()
            running_loss += loss.item()

        return running_loss / max(1, len(self.training_loader))

    def calc_auroc_aupr_on_indexes(self, model, validation_dataset, batch_size=20, pad_value=-5000.0):

        model.eval()
        all_scores, all_labels = [], []
        batched_data = batch(validation_dataset, n=batch_size)

        with torch.no_grad():
            for one_batch in tqdm(batched_data, total= len(validation_dataset)/batch_size, desc="Calculating AUC"):

                # Gather items: dataset[i] -> (binder_emb [L,D], target_emb [L,D], label)
                items = [i for i in one_batch]
                binders = torch.stack([binder_emb for (binder_emb, _, _) in items]).to(self.device)
                targets = torch.stack([target_emb for (_, target_emb, _) in items]).to(self.device)
                labels = np.array([float(lbl) for *_, lbl in items], dtype=np.float32)

                # Forward: logits per pair [B]
                logits = model.forward(binders, targets).detach().cpu().numpy()

                all_scores.extend(logits.tolist())
                all_labels.extend(labels.tolist())

        all_scores = np.array(all_scores, dtype=np.float64)
        all_labels = np.array(all_labels, dtype=np.int64)

        fpr, tpr, _ = metrics.roc_curve(all_labels, all_scores)
        auroc = metrics.roc_auc_score(all_labels, all_scores)
        aupr  = metrics.average_precision_score(all_labels, all_scores)

        return auroc, aupr, fpr, tpr
    
    def validate(self, dataloader, indexes_for_auc=False, auROC_dataset=False):
        self.model.eval()
        running_loss, n_loss = 0.0, 0
        all_scores, all_labels = [], []
    
        with torch.no_grad():
            for batch_data in tqdm(dataloader, total=len(dataloader), desc="Validation"):
                loss, logits, labels = self.model.validation_step(batch_data, self.device)
                running_loss += float(loss)
                n_loss += 1
    
                # expect logits [B], labels [B] on device; move once
                all_scores.append(logits.detach().float().cpu())
                all_labels.append(labels.detach().long().cpu())
    
        val_loss = running_loss / max(1, n_loss)
    
        if all_scores:
            scores = torch.cat(all_scores).numpy()
            labs   = torch.cat(all_labels).numpy()
            val_auroc = metrics.roc_auc_score(labs, scores)
            val_aupr  = metrics.average_precision_score(labs, scores)
        else:
            val_auroc = float("nan")
            val_aupr  = float("nan")
    
        return val_loss, val_auroc, val_aupr

    def train_model(self):

        if self.verbose:
            print(f"Training model {str(self.runID)}")
        
        # Pre-training snapshot
        val_loss, val_auroc, val_aupr  = self.validate(
            dataloader=self.validation_loader, 
            indexes_for_auc=self.test_indexes_for_auROC, 
            auROC_dataset=self.test_dataset)

        if self.verbose:
            print(
                f'Before training - Val Loss {val_loss:.4f} | '
                f'Val AUROC {val_auroc if val_auroc==val_auroc else float("nan"):.4f} | '
                f'Val AUPR {val_aupr if val_aupr==val_aupr else float("nan"):.4f}'
            )
            
        if self.wandb_tracker:
            log_items = {
                "Val Loss": val_loss,
                "Val AUROC": val_auroc,
                "Val AUPR": val_aupr,
            }

            self.wandb_tracker.log(log_items)
            
        # --- Epoch loop ---
        for epoch in tqdm(range(1, self.EPOCHS + 1), total=self.EPOCHS, desc="Epochs"):
            torch.cuda.empty_cache()
            train_loss = self.train_one_epoch()
    
            val_loss, val_auroc, val_aupr = self.validate(
                dataloader=self.validation_loader,
                indexes_for_auc=self.test_indexes_for_auROC,
                auROC_dataset=self.test_dataset,
            )


            torch.cuda.empty_cache()
            # Save checkpoints periodically
            # if self.model_save_steps and (epoch % self.model_save_steps == 0):
            #     if epoch in [5, 10, 15]:
            #         check_point_folder = os.path.join(self.trained_model_dir, f"{self.split_id}_{str(self.runID)}_epochs_{epoch}")
            #         if self.verbose:
            #             print("Saving model to:", check_point_folder)
            #         os.makedirs(check_point_folder, exist_ok=True)
            #         checkpoint_path = os.path.join(
            #             check_point_folder, f"{str(self.runID)}_checkpoint_epoch_{epoch}.pth"
            #         )
            #         torch.save(
            #             {
            #                 'model_state_dict': self.model.state_dict(),
            #                 'optimizer_state_dict': self.optimizer.state_dict(),
            #                 'val_loss': val_loss,
            #             },
            #             checkpoint_path,
            #         )
    
            if self.verbose and (epoch % self.print_frequency_loss == 0):
                print(
                    f'EPOCH {epoch} - Train Loss {train_loss:.4f} | '
                    f'Val Loss {val_loss:.4f} | Val AUROC {val_auroc if val_auroc==val_auroc else float("nan"):.4f} | '
                    f'Val AUPR {val_aupr if val_aupr==val_aupr else float("nan"):.4f}'
                )
    
            if self.wandb_tracker:
                log_items = {
                    "Train Loss": train_loss,
                    "Val Loss": val_loss,
                    "Val AUROC": val_auroc,
                    "Val AUPR": val_aupr,
                }

                self.wandb_tracker.log(log_items)
    
        if self.wandb_tracker:
            self.wandb_tracker.finish()

In [87]:
batch_size = 20
learning_rate = 2e-5
EPOCHS = 15

# login once (env var preferred)
# if use_wandb:
#     import wandb
#     wandb.login()

model = MiniCLIP_w_transformer_crossattn()
optimizer = AdamW(model.parameters(), lr=learning_rate)

# loaders
train_loader = train_loader
val_loader   = val_loader

# accelerator
accelerator = Accelerator()
device = accelerator.device
model, optimizer, train_loader, val_loader = accelerator.prepare(model, optimizer, train_loader, val_loader)

# wandb
if use_wandb:
    run = wandb.init(
        project="CLIP_PPint_metaanalysis",
        name=f"class_weighting_CVsplit1{runID}",
        # group="cv_splits",
        config={"learning_rate": learning_rate, "batch_size": batch_size, "epochs": EPOCHS,
                "architecture": "MiniCLIP_w_transformer_crossattn", "dataset": "Meta analysis"},
    )
    wandb.watch(accelerator.unwrap_model(model), log="all", log_freq=100)
else:
    run = None

# train
training_wrapper = TrainWrapper_MetaAnal(
    model=model,
    training_loader=train_loader,
    validation_loader=val_loader,
    test_dataset=val_binders,   # ok if you truly want “full val”
    optimizer=optimizer,
    EPOCHS=EPOCHS,
    runID=runID,
    device=device,
    model_save_steps=model_save_steps,
    model_save_path=trained_model_dir,
    v=True,
    wandb_tracker=run,
    split_id=i+1
)

training_wrapper.train_model()

# cleanup between splits
if use_wandb:
    wandb.finish()
del training_wrapper, model, optimizer, train_loader, val_loader
if torch.cuda.is_available():
    torch.cuda.empty_cache()
try:
    accelerator.free_memory()
except AttributeError:
    pass

Training model 5661d4ea-f6d4-43d1-89cd-77e285870386


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:15<00:00,  4.51it/s]


Before training - Val Loss 9.7293 | Val AUROC 0.3603 | Val AUPR 0.0841


Running through epoch: 100%|████████████████████████████████████████████████████████████████████████████████| 40/40 [00:21<00:00,  1.89it/s]

Epochs:   7%|██████▍                                                                                         | 1/15 [00:37<08:47, 37.66s/it]

EPOCH 1 - Train Loss 5.4628 | Val Loss 9.4804 | Val AUROC 0.3557 | Val AUPR 0.0834


...


Running through epoch: 100%|████████████████████████████████████████████████████████████████████████████████| 40/40 [00:20<00:00,  1.92it/s]

Epochs:  93%|████████████████████████████████████████████████████████████████████████████████████████▋      | 14/15 [08:42<00:37, 37.29s/it]

EPOCH 14 - Train Loss 2.0499 | Val Loss 3.4097 | Val AUROC 0.3241 | Val AUPR 0.0791



Running through epoch: 100%|████████████████████████████████████████████████████████████████████████████████| 40/40 [00:21<00:00,  1.90it/s]

Epochs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [09:19<00:00, 37.30s/it]

EPOCH 15 - Train Loss 1.9016 | Val Loss 2.9213 | Val AUROC 0.3245 | Val AUPR 0.0790


Train Loss,██▇▇▇▆▅▅▅▄▃▂▂▁▁
Val AUPR,█▇▆▆▅▅▄▄▃▃▂▂▂▁▁▁
Val AUROC,█▇▆▆▅▅▄▄▃▃▂▂▁▁▁▁
Val Loss,██▇▇▇▆▆▆▅▄▄▃▃▂▂▁
Train Loss,1.90159
Val AUPR,0.07895
Val AUROC,0.32452
Val Loss,2.92126


`validation_dataset` / `training_dataset` store embedding of binder, embedding of target and label (binder - 1, non-binder - 0)